https://www.superdatascience.com/pages/deep-learning -> course documents
This is actually a movie created by a RNN. Additional reading: https://arstechnica.com/gaming/2016/06/an-ai-wrote-this-movie-and-its-strangely-moving/

Reference: http://colah.github.io/posts/2015-08-Understanding-LSTMs/

The goal is to predict the trend of the Google Stock price

In [1]:
# Importing the Libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [17]:
# Part 1 - Data Preprocessing
# The RNN will only be trained on the training set. It will not see the test set until after the training is done!
dataset_train = pd.read_csv('dataset/Google_Stock_Price_Train.csv')
training_set = dataset_train.iloc[:,1:2].values # creates a data frame
    # Do not want just a vector array. Want a 2d array for np. This means we must import an array for the number of columns.
    # The trick is to reference 1:2 because in python the upperbound is excluded. so only column index[1] is taken but imported
    # in the correct format (numpy array of 1 column)
    # the .values makes it a np array
print(type(dataset_train))
print(type(training_set))

# Feature Scaling
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0,1), copy=True)
#fit means that it will get the min and max and will scale the data. The transform will compute the scaled (normalized) value based on the normalization formula.
training_set_scaled = sc.fit_transform(training_set)
#print(training_set_scaled)

# Creating a data structure with 60 timesteps and 1 output
# the 60 timestep means that the model will look at time t, the previous 60 times steps and time step t. Based on those values
# the model will try to predict the next output. 
# The value of 60 timesteps was chosen by trial and error. There are ~20 buisness days in a month so 60 timesteps is 60 buisness days (3 months)
X_train = []
y_train = []
#print(len(training_set_scaled))

for i in range(60, len(training_set_scaled)):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])

# y_train is the array that needs to be predicted based on the data in X_train
X_train, y_train = np.array(X_train), np.array(y_train)

print(X_train)
print("")
print("Break for viewing")
print("")
print(y_train)

# Reshaping
# the 2nd argument is the new shape we want the array to have
# want 3 dimensions for X_train with the 3rd corresponding to the indicator (prediction)
# Information for the recurrent input shapes: https://keras.io/layers/recurrent/
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1]), 1)
    # 1 corresponds to the Google Open Stock Price

<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
[[0.08581368 0.09701243 0.09433366 ... 0.07846566 0.08034452 0.08497656]
 [0.09701243 0.09433366 0.09156187 ... 0.08034452 0.08497656 0.08627874]
 [0.09433366 0.09156187 0.07984225 ... 0.08497656 0.08627874 0.08471612]
 ...
 [0.92106928 0.92438053 0.93048218 ... 0.95475854 0.95204256 0.95163331]
 [0.92438053 0.93048218 0.9299055  ... 0.95204256 0.95163331 0.95725128]
 [0.93048218 0.9299055  0.93113327 ... 0.95163331 0.95725128 0.93796041]]

Break for viewing

[0.08627874 0.08471612 0.07454052 ... 0.95725128 0.93796041 0.93688146]


# Part 2 - Building the RNN
    # Building a LSTM
    # Look up CTC

In [18]:
# Part 2 - Building the RNN
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

Using TensorFlow backend.


In [22]:
# Initialising the RNN
regressor = Sequential()
    # represents a sequence of layers

# Adding the first LSTM layer and some Dropout regularisation (added to avoid overfitting). Don't want overfitting in general
# LSTM first layer. Need to use the LSTM class. Need to input 3 arguments
    # number of units = number of LSTM memory cells to have in the layer. Like neurons
    # 2nd arg. return sequences. Set to true because building a stacked LSTM. So have to return values if having multiple LSTM layers
    # 3rd arg. input shape. The shape of the input (e.g. shape of X_train which is 3D). Only have to specify 2 of the 3 dimensions though
        # X_train.shape[1] is the time steps
        # 1 corresponds to the indicators (predictors)
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1) ))

# Adding dropout regularisation
    # standard to dropout 20%. So 20% of the neurons will be ignored in the backward and forward propegation (so 10 neurons)
    # these are ignored each iteration of the training
regressor.add(Dropout(0.2))



TypeError: while_loop() got an unexpected keyword argument 'maximum_iterations'

In [ ]:
# Part 3 - Making the predictions and visulising the results